# **Create Betting Odds Dataset**

## *Import Odds Data & Needed Functions*

Packages Imports

In [1]:
# Import needed packages
import pandas as pd
import numpy as np
import json
import collections
from tqdm.notebook import tqdm
import re
# Change Pandas rows and columns' options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

##### Generic Import Functions

In [2]:
def flatten(d, parent_key='', sep='_'): 
    """This function turns a nested dictionary into a flattened dictionary.
    d: nested dictionary
    """
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [3]:
def keep_n_mostcommon_odds(jsonfilepath_or_object, n_tokeep):
    """This function takes either a JSON file or a Python object and keeps only bets that appear more than a certain threshold value (n_tokeep), storing them in a list of dictionaries.
    jsonfilepath_or_object: JSON file path or Python object
    n_tokeep: threshold value for features to keep 
    """
    # If jsonfilepath_or_object is a file path
    if isinstance(jsonfilepath_or_object, str):
        f = open(jsonfilepath_or_object)
        odds_data = json.load(f)
        f.close()
    # If jsonfilepath_or_object is a python object 
    elif isinstance(jsonfilepath_or_object, object):
        odds_data = jsonfilepath_or_object
    # Obtain features and counts
    feat = [key for fix in odds_data for key in fix.keys()]
    feat_count = collections.Counter(feat).most_common()
    frequent_feats = {x[0] for x in feat_count if x[1] > n_tokeep}
    new_odds = [{k: fix[k] for k in frequent_feats if k in fix} for fix in odds_data]
    # Check 
    check = {key for i in new_odds for key in i.keys()}
    if check == frequent_feats:
        print('Check for correct keys: OK', '\nFeatures with at least ', n_tokeep, 'observations: ', len(frequent_feats))
    else: 
        print('Check for correct keys: ERROR') 
    return new_odds

##### Specific Import & Preparation Functions

In [4]:
def odds_data_import(jsonpath_or_object):
    """This function imports odds data from a JSON file (using keep_n_mostcommon_odds()). It also convert the resulting dictionary into a DataFrame, and excecutes some basic pre-cleaning tasks.
    jsonpath_or_object: JSON file path or Python object
    """
    # Import JSON data as dictionary + Convert dictionary to DataFrame (setting id as index)
    odds_dict = keep_n_mostcommon_odds(jsonpath_or_object, 1)
    odds_df = pd.DataFrame.from_dict(odds_dict)
    odds_df.set_index('id', inplace=True)
    # Drop both columns and rows that contain exclusively NAs + strip columns' names
    odds_df.dropna(axis=1, how='all', inplace=True)
    odds_df.dropna(axis=0, how='all', inplace=True)
    odds_df.columns = odds_df.columns.str.strip()
    # Print some info
    print('Odds DataFrame shape: ', odds_df.shape)
    return odds_df

In [5]:
duplicates_rename = {'HndRES_MyBet__2 :':['HndRES_MyBetone__2','HndRES_MyBettwo__2'],'HndRES_MyBet__X :':['HndRES_MyBetone__X','HndRES_MyBettwo__X'],'HndRES_WinLineBet__1 :':['HndRES_WinLineBetone__1','HndRES_WinLineBettwo__1'],'HndRES_WinLineBet__2 :':['HndRES_WinLineBetone__2','HndRES_WinLineBettwo__2'],'HndRES_MyBet__1 :':['HndRES_MyBetone__1','HndRES_MyBettwo__1'],'HndRES_WinLineBet__X :':['HndRES_WinLineBetone__X','HndRES_WinLineBettwo__X'],'O/U_bet365__Over':['O/U_bet365one__Over','O/U_bet365two__Over'],'AlternativeTG_bet365__Over':['AlternativeTG_bet365one__Over','AlternativeTG_bet365two__Over']}

In [6]:
def odd_preparation(odd):
    """This function excecutes some cleaning tasks: dealing with duplicates columns and renaming columns for next steps.
    odd: DataFrame containg odds data
    """
    # Get dictionary to rename columns
    global duplicates_rename
    # Check: print duplicates columns before and after renaming process
    print('Duplicates Features before renaming: ', odd.columns[odd.columns.duplicated()].tolist())
    odd = odd.rename(columns=lambda c: duplicates_rename[c].pop(0) if c in duplicates_rename.keys() else c)
    print('Duplicates Features after renaming: ', odd.columns[odd.columns.duplicated()].tolist())
    # Rename columns in order to automate next step
    odd.columns = odd.columns.str.replace(r'^(3W.*_.+)__Draw$', lambda x: x.group(1) + 'a__X', regex=True)
    odd.columns = odd.columns.str.replace(r'byGamesinMatch', '_', regex=True) 
    odd.columns = odd.columns.str.replace(r'^(TeamToScoreFirst_.+)__No Goals$', lambda x: x.group(1) + 'a__X', regex=True)
    odd.columns = odd.columns.str.replace(r'^(AH[^Cards].*)__Home$', lambda x: x.group(1) + 'a__1', regex=True)
    odd.columns = odd.columns.str.replace(r'^(AH[^Cards].*)__Away$', lambda x: x.group(1) + 'a__2', regex=True)
    odd.columns = odd.columns.str.replace(r'\\(Games\\)', '', regex=True)
    odd.columns = odd.columns.str.replace(r'^WintoNil-H_(.+)__Yes', lambda x: 'WinToNil_' + x.group(1) + 'a__1', regex=True)
    odd.columns = odd.columns.str.replace(r'^WintoNil-A_(.+)__Yes', lambda x: 'WinToNil_' + x.group(1) + 'a__2', regex=True)
    odd.columns = odd.columns.str.replace(r'^Hwinbothhalves_(.+)__Yes', lambda x: 'WinBothHalves_'+ x.group(1) +'a__1', regex=True)
    odd.columns = odd.columns.str.replace(r'^Awinbothhalves_(.+)__Yes', lambda x: 'WinBothHalves_'+ x.group(1) +'a__2', regex=True)
    odd.columns = odd.columns.str.replace(r'^(TeamToScoreLast.*)__No goal', lambda x: x.group(1) + 'a__X', regex=True)
    odd.columns = odd.columns.str.replace(r'^(HighestScoringHalf.*)__Tie', lambda x: x.group(1) + 'a__X', regex=True)
    odd.columns = odd.columns.str.replace(r'^(CleanSheet.*)__(1|2)', lambda x: x.group(1) + 'a__Yes', regex=True)
    odd.columns = odd.columns.str.replace(r'(^CS.*_.+)__.*(\d:\d).*$', lambda x: x.group(1) + 'a__' + x.group(2), regex=True)
    odd.columns = odd.columns.str.replace(r'^(HndRES.*_.+)__Tie', lambda x: x.group(1) + 'a__X', regex=True)
    return odd

##### New Features (Summary Statistics) Functions

In [7]:
def new_features_from_odds(dFrame, odd_type , odd_result):
    """This function creates new features for odds data by computing summary statistics for columns filtered using a regular expression (given the odd types and possible results).
    dFrame: DataFrame containg odds data
    odd_type: odd type to filter for 
    odd_result: possible results linked with the specific odd type
    """
    # Create regex and filter data using it
    my_regex = r"^" + re.escape(odd_type)  + r".+" + re.escape(odd_result) + r"$"
    mask = dFrame.loc[:, dFrame.filter(regex=my_regex).columns.tolist()]
    # Create new features as summary statistics of the filtered data
    dFrame[odd_type + odd_result +'_count'] =  mask.count(axis='columns', numeric_only=True)
    dFrame[odd_type + odd_result +'_mean'] =  mask.mean(axis=1, skipna=True)
    dFrame[odd_type + odd_result + '_median'] =  mask.median(axis=1, skipna=True)
    dFrame[odd_type + odd_result + '_std'] =  mask.std(axis=1, skipna=True)
    dFrame[odd_type + odd_result + '_var'] =  mask.var(axis=1, skipna=True)
    dFrame[odd_type + odd_result + '_max'] =  mask.max(axis=1, skipna=True)
    dFrame[odd_type + odd_result + '_min'] =  mask.min(axis=1, skipna=True)
    dFrame[odd_type + odd_result + '_1st_quart'] = mask.quantile(q=0.25, axis=1)
    dFrame[odd_type + odd_result + '_3rd_quart'] = mask.quantile(q=0.75, axis=1)
    dFrame[odd_type + odd_result + '_range'] = dFrame[odd_type + odd_result + '_max'] -  dFrame[odd_type + odd_result + '_min']
    dFrame[odd_type + odd_result + '_IQR'] = dFrame[odd_type + odd_result + '_3rd_quart'] - dFrame[odd_type + odd_result + '_1st_quart']
    return dFrame

In [8]:
def odds_columns_checks(dataframe, print_list=False, complete=False):
    """This function checks the data quality and prints some messages related to: DataFrame shape, number of duplicates (if any), columns' data types. In case of duplicates columns the function groups together columns with the same name. 
    dataframe: DataFrame to check
    print_list (Default: False): if True the function prints duplicates columns' names
    complete (Default: False): if True the function provides more extensive information
    """
    # Print Shape of DataFrame and compute if there are any duplicates columns
    print('Odds Summary Statistics DataFrame Shape: ', dataframe.shape)
    no_duplicates = (sum(dataframe.columns.duplicated().tolist()) == 0)
    # If complete is True print the entire check 
    if complete:
        print('No Duplicated columns: ', no_duplicates)
        print('All Columns Data Types are Numeric: ', len(dataframe.dtypes[dataframe.dtypes == 'object']) == 0)
    if no_duplicates is False and complete is True:
        print('N. Total Columns: ', len(dataframe.columns.tolist()), '\nN. Unique Columns: ', len(set(dataframe.columns.tolist())))
        print('N. Duplicates: ', len(dataframe.columns[dataframe.columns.duplicated()].tolist()))
        if print_list:
            print('Duplicates: ', dataframe.columns[dataframe.columns.duplicated()].tolist())
        dataframe = dataframe.groupby(by=dataframe.columns, axis=1).mean()
        print('After Duplicates Cleaning - N. Duplicates: ', len(dataframe.columns[dataframe.columns.duplicated()].tolist()))
    # If complete is False do not print entire check, just change dataframe
    elif no_duplicates is False and complete is False:
        dataframe = dataframe.groupby(by=dataframe.columns, axis=1).mean()
    return dataframe

In [9]:
def new_odds_features_df(initial_df, key, types_list, results_list):
    """This function filters initial_df's columns using key as regular expression, drops rows containing only NAs, converts columns type to numeric, checks the data quality (using odds_columns_checks()), and creates new features (using new_features_from_odds).
    initial_df: DataFrame
    key: string used to filter by column name
    types_list: list of odd types to consider
    results_list: list of possible results for types_list
    """
    # Filtering data: consider only columns compatible with regex (key)
    df_temp = initial_df.loc[:, initial_df.filter(regex=key).columns.tolist()]
    # Drop rows with only NAs + Convert columns' dtypes to numeric
    df_temp.dropna(axis=0, how='all', inplace=True)
    df_temp = df_temp.apply(pd.to_numeric, errors='coerce', downcast='float')
    # Check results
    df_temp = odds_columns_checks(df_temp)
    n_features = df_temp.shape[1]
    # Create new features using new_features_from_odds()
    for ty in types_list:
        for res in results_list:
            df_temp = new_features_from_odds(dFrame=df_temp, odd_type=ty, odd_result=res)
    return df_temp.iloc[:, (n_features):]

In [10]:
def special_new_features_df(df_filtered, types_list, results_list):
    """This function is a simplified version of new_odds_features_df() needed to better deal with problematic data. It differs because it does not filter the given data, requiring an already filtered DataFrame. 
    In common with new_odds_features_df(), this function drops rows containing only NAs, converts columns type to numeric, checks the data quality (using odds_columns_checks()), and creates new features (using new_features_from_odds).
    df_filtered: already filtered DataFrame
    types_list: list of odd types to consider
    results_list: list of possible results for types_list
    """
    # Drop rows with only NAs + Convert columns' dtypes to numeric
    df_filtered.dropna(axis=0, how='all', inplace=True)
    df_filtered = df_filtered.apply(pd.to_numeric, errors='coerce', downcast='float')
    # Check results
    df_filtered = odds_columns_checks(df_filtered)
    n_features = df_filtered.shape[1]
    # Create new features using new_features_from_odds()
    for ty in types_list:
        for res in results_list:
            df_filtered = new_features_from_odds(dFrame=df_filtered, odd_type=ty, odd_result=res)
    return df_filtered.iloc[:, (n_features):]

## *Create Odds Dataset*

In [11]:
# List of dictionaries containg all the needed odd types 
complete_oddtypes = [{'regex_odd_type': '^3W', 'types': ['3W_', '3W1stHalf_', '3W2ndHalf_'], 'results': ['_1', '_X', '_2']}, {'regex_odd_type': '^O/U', 'types': ['O/U_', 'O/U1stHalf_', 'O/U2ndHalf_'], 'results': ['_Over', '_Under']}, {'regex_odd_type': '^Corners1x2_', 'types': ['Corners1x2_'], 'results':['_1', '_X', '_2']}, {'regex_odd_type': '^TeamToScoreFirst_', 'types': ['TeamToScoreFirst_'], 'results': ['_1', '_X', '_2']}, {'regex_odd_type': '^H/A_', 'types': ['H/A'], 'results': ['_1', '_2']}, {'regex_odd_type': 'ScoreaGoal_', 'types': ['HTeamScoreaGoal_', 'ATeamScoreaGoal_'], 'results': ['_No', '_Yes']}, {'regex_odd_type': '^BothTeamsToScore', 'types': ['BothTeamsToScore_', 'BothTeamsToScore-1stHalf_', 'BothTeamsToScore-2ndHalf_'], 'results': ['_No', '_Yes']}, {'regex_odd_type': '^Odd/Even', 'types': ['Odd/Even_', 'Odd/Even1stHalf_'], 'results': ['_Even', '_Odd']}, {'regex_odd_type': '^CornersOU.*(Over|Under)$', 'types': ['CornersOU_'], 'results': ['_Over', '_Under']}, {'regex_odd_type': '^AH[^Cards].*_(1|2|Home|Away)$', 'types': ['AH_', 'AHFirstHalf_'], 'results': ['_1', '_2']}, {'regex_odd_type': '^Total-(H|A)_', 'types': ['Total-H_', 'Total-A_'], 'results': ['_Over', '_Under']}, {'regex_odd_type': '^(WinToNil|WintoNil).*(_1|_2|_Yes)$', 'types': ['WinToNil_'], 'results': ['_1', '_2']}, {'regex_odd_type': '^((A|H)winbothhalves_|WinBothHalves_).*(_1|_2|_Yes)', 'types': ['WinBothHalves_'], 'results': ['_1', '_2']}, {'regex_odd_type': '^TeamToScoreLast.*_(1|2|No goal|X)', 'types': ['TeamToScoreLast_'], 'results': ['_1', '_2', '_X']}, {'regex_odd_type': '^HighestScoringHalf_', 'types': ['HighestScoringHalf_'], 'results': ['_1st Half', '_2nd Half', '_X']}, {'regex_odd_type': '^CleanSheet', 'types': ['CleanSheet-H_', 'CleanSheet-A_'], 'results': ['_Yes', '_No']}, {'regex_odd_type': '^ToWinEitherHalf', 'types': ['ToWinEitherHalf_'], 'results': ['_1','_2']}, {'regex_odd_type': '^CS', 'types': ['CS_', 'CS1stHalf_', 'CS2ndHalf_'], 'results': ['_0:0', '_1:0', '_0:1', '_2:0', '_0:2', '_3:0', '_0:3', '_4:0', '_0:4', '_5:0', '_0:5', '_6:0', '_0:6', '_7:0', '_0:7', '_1:1', '_2:1', '_1:2', '_3:1', '_1:3', '_4:1', '_1:4', '_5:1', '_1:5', '_6:1', '_1:6', '_2:2', '_3:2', '_2:3', '_4:2', '_2:4', '_5:2', '_2:5', '_6:2', '_2:6',  '_3:3', '_4:3', '_3:4', '_4:4', '_5:3', '_3:5']}, {'regex_odd_type': '^HndRES.*__(1|2|X|Tie)$', 'types': ['HndRES_', 'HndRES1stHalf_'], 'results': ['_1', '_X', '_2']}]

Leagues odds data import & preparation

In [12]:
leagues_odds_df = odds_data_import(jsonpath_or_object='../../Data/From_Collection/Match&Odds/leagues_odds.json')
league_odds_clean = odd_preparation(odd=leagues_odds_df)

Check for correct keys: OK 
Features with at least  1 observations:  84056
Odds DataFrame shape:  (8916, 84055)
Duplicates Features before renaming:  ['HndRES_MyBet__X :', 'HndRES_MyBet__2 :', 'HndRES_WinLineBet__1 :', 'HndRES_WinLineBet__2 :', 'HndRES_MyBet__1 :', 'HndRES_WinLineBet__X :', 'O/U_bet365__Over', 'AlternativeTG_bet365__Over']
Duplicates Features after renaming:  []


Double-Chance odd type + Exact Goals odd type

In [13]:
DC_dict = {'types': ['DoubleChance_', 'DoubleChance-1stHalf_', 'DoubleChance-2ndHalf_'], 'results': ['_1X', '_12', '_X2']}
# Filter data using regex 
df_DC = league_odds_clean.loc[:, league_odds_clean.filter(regex='^DoubleChance').columns.tolist()]
# Rename columns
df_DC.columns = df_DC.columns.str.replace('/', '')
df_DC.columns = df_DC.columns.str.replace(r'__.+ or Draw$', '__1X', regex=True)
df_DC.columns = df_DC.columns.str.replace(r'__Draw or .*$', '__X2', regex=True)
# Get new features using special_new_features_df()
df_DC_new = special_new_features_df(df_filtered=df_DC, types_list=DC_dict['types'], results_list=DC_dict['results'])
# Check new DataFrame frequency columns by odd type
df_DC_new[df_DC_new.filter(regex='count').columns.tolist()].describe()

Odds Summary Statistics DataFrame Shape:  (8910, 3017)


,DoubleChance__1X_count,DoubleChance__12_count,DoubleChance__X2_count,DoubleChance-1stHalf__1X_count,DoubleChance-1stHalf__12_count,DoubleChance-1stHalf__X2_count,DoubleChance-2ndHalf__1X_count,DoubleChance-2ndHalf__12_count,DoubleChance-2ndHalf__X2_count
count,8910.000000,8910.000000,8910.000000,8910.000000,8910.000000,8910.000000,8910.000000,8910.000000,8910.000000
mean,11.793266,11.443996,11.854994,7.591021,7.234343,7.611111,3.990011,3.998878,3.999102
std,3.818611,3.406918,3.769076,5.342650,4.956435,5.348192,3.170610,3.170466,3.170466
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9.000000,9.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,11.000000,11.000000,11.000000,8.000000,9.000000,9.000000,4.000000,4.000000,4.000000
75%,16.000000,15.000000,16.000000,13.000000,12.000000,13.000000,7.000000,7.000000,7.000000
max,19.000000,18.000000,19.000000,16.000000,15.000000,16.000000,9.000000,9.000000,9.000000


In [14]:
eg_diz = {'types': ['ExactGoalsNumber_','HTeamExactGoalsNumber_','ATeamExactGoalsNumber_','2ndHalfExactGoalsNumber_','1stHalfExactGoalsNumber_'], 'results': ['_0', '_1', '_2', '_3', '_4', '_5', '_6', '_7']}
# Filter data using regex
eg = league_odds_clean.loc[:, league_odds_clean.filter(regex='ExactGoalsNumber_').columns.tolist()]
# Rename columns
eg.columns = eg.columns.str.replace(r'^(.*)_(.+)__.*(\d).*$',lambda x: x.group(1) +'_'+ x.group(2) +'a__'+ x.group(3), regex=True)
# Get new features using special_new_features_df()
df_exactgoals_new = special_new_features_df(df_filtered=eg, types_list=eg_diz['types'], results_list=eg_diz['results'])
# Check new DataFrame frequency columns by odd type
df_exactgoals_new[df_exactgoals_new.filter(regex='count').columns.tolist()].describe()

Odds Summary Statistics DataFrame Shape:  (8746, 1966)


,ExactGoalsNumber__0_count,ExactGoalsNumber__1_count,ExactGoalsNumber__2_count,ExactGoalsNumber__3_count,ExactGoalsNumber__4_count,ExactGoalsNumber__5_count,ExactGoalsNumber__6_count,ExactGoalsNumber__7_count,HTeamExactGoalsNumber__0_count,HTeamExactGoalsNumber__1_count,HTeamExactGoalsNumber__2_count,HTeamExactGoalsNumber__3_count,HTeamExactGoalsNumber__4_count,HTeamExactGoalsNumber__5_count,HTeamExactGoalsNumber__6_count,HTeamExactGoalsNumber__7_count,ATeamExactGoalsNumber__0_count,ATeamExactGoalsNumber__1_count,ATeamExactGoalsNumber__2_count,ATeamExactGoalsNumber__3_count,ATeamExactGoalsNumber__4_count,ATeamExactGoalsNumber__5_count,ATeamExactGoalsNumber__6_count,ATeamExactGoalsNumber__7_count,2ndHalfExactGoalsNumber__0_count,2ndHalfExactGoalsNumber__1_count,2ndHalfExactGoalsNumber__2_count,2ndHalfExactGoalsNumber__3_count,2ndHalfExactGoalsNumber__4_count,2ndHalfExactGoalsNumber__5_count,2ndHalfExactGoalsNumber__6_count,2ndHalfExactGoalsNumber__7_count,1stHalfExactGoalsNumber__0_count,1stHalfExactGoalsNumber__1_count,1stHalfExactGoalsNumber__2_count,1stHalfExactGoalsNumber__3_count,1stHalfExactGoalsNumber__4_count,1stHalfExactGoalsNumber__5_count,1stHalfExactGoalsNumber__6_count,1stHalfExactGoalsNumber__7_count
count,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.0,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.0,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.0,8746.0,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.000000,8746.0,8746.0
mean,4.945918,4.780014,4.777384,4.776698,4.777727,4.315687,4.011319,2.002401,4.048594,3.879373,4.065516,3.719186,1.965127,0.233135,0.001944,0.0,3.586097,3.420764,3.600960,3.242511,1.468900,0.205237,0.026298,0.0,1.924537,1.944775,1.943517,1.862566,1.108392,0.798422,0.0,0.0,3.971987,3.801967,3.989138,3.869426,1.484907,0.798422,0.0,0.0
std,3.153178,2.971886,2.972709,2.972658,2.973139,2.503957,2.244767,1.002624,2.782004,2.653449,2.809961,2.699857,1.976777,0.422852,0.044048,0.0,2.671144,2.579144,2.704957,2.663319,2.076525,0.436814,0.164955,0.0,1.165062,1.183280,1.183076,1.141293,0.706520,0.401201,0.0,0.0,2.687202,2.599842,2.716865,2.700462,0.961257,0.401201,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0
50%,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,2.000000,4.000000,4.000000,4.000000,4.000000,2.000000,0.000000,0.000000,0.0,3.000000,3.000000,3.000000,3.000000,1.000000,0.000000,0.000000,0.0,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,0.0,0.0,4.000000,4.000000,4.000000,4.000000,2.000000,1.000000,0.0,0.0
75%,8.000000,8.000000,8.000000,8.000000,8.000000,6.000000,6.000000,3.000000,7.000000,6.000000,7.000000,7.000000,2.000000,0.000000,0.000000,0.0,6.000000,5.000000,6.000000,6.000000,1.000000,0.000000,0.000000,0.0,3.000000,3.000000,3.000000,3.000000,2.000000,1.000000,0.0,0.0,7.000000,6.000000,7.000000,7.000000,2.000000,1.000000,0.0,0.0
max,10.000000,10.000000,10.000000,10.000000,10.000000,8.000000,8.000000,4.000000,9.000000,9.000000,9.000000,9.000000,6.000000,1.000000,1.000000,0.0,9.000000,9.000000,9.000000,9.000000,6.000000,2.000000,2.000000,0.0,4.000000,4.000000,4.000000,4.000000,2.000000,1.000000,0.0,0.0,9.000000,9.000000,9.000000,9.

All Odds Types DataFrame

In [15]:
# Merge Double-Chance dataframe and Exact Goals DataFrame
df_complete_odds = df_DC_new.merge(df_exactgoals_new, how='outer', left_index=True, right_index=True)
somma_cols = df_complete_odds.shape[1] # for later check

In [16]:
# Loop over complete_oddtypes list of dictionaries, creating a temporary DataFrame for each list element and merging it one-by-one to the main DataFrame (df_complete_odds)

for single_oddtype in tqdm(complete_oddtypes):
    # Create new features temporary DataFrame
    df_temporary = new_odds_features_df(initial_df=league_odds_clean, key=single_oddtype['regex_odd_type'], types_list=single_oddtype['types'], results_list=single_oddtype['results'])
    # print(df_temporary.shape)
    somma_cols += df_temporary.shape[1] # for later check
    # Merge temporary DataFrame with existing df_complete_odds DataFrame
    df_complete_odds = df_complete_odds.merge(df_temporary, how='outer', left_index=True, right_index=True)

  0%|          | 0/19 [00:00<?, ?it/s]

Odds Summary Statistics DataFrame Shape:  (8915, 334)
Odds Summary Statistics DataFrame Shape:  (8915, 252)
Odds Summary Statistics DataFrame Shape:  (7438, 68)
Odds Summary Statistics DataFrame Shape:  (8911, 175)
Odds Summary Statistics DataFrame Shape:  (8906, 72)
Odds Summary Statistics DataFrame Shape:  (8743, 46)
Odds Summary Statistics DataFrame Shape:  (8913, 150)
Odds Summary Statistics DataFrame Shape:  (7969, 76)
Odds Summary Statistics DataFrame Shape:  (8688, 35)
Odds Summary Statistics DataFrame Shape:  (8915, 119)
Odds Summary Statistics DataFrame Shape:  (8745, 74)
Odds Summary Statistics DataFrame Shape:  (8764, 33)
Odds Summary Statistics DataFrame Shape:  (8764, 30)
Odds Summary Statistics DataFrame Shape:  (8910, 58)
Odds Summary Statistics DataFrame Shape:  (8745, 52)
Odds Summary Statistics DataFrame Shape:  (8741, 29)
Odds Summary Statistics DataFrame Shape:  (6982, 16)
Odds Summary Statistics DataFrame Shape:  (8751, 2472)
Odds Summary Statistics DataFrame Shape

Checks for merged data

In [17]:
# Checks
print('Check for correct number of columns: ', somma_cols == df_complete_odds.shape[1])
df_complete_odds = odds_columns_checks(df_complete_odds, print_list=True, complete=True)

Check for correct number of columns:  True
Odds Summary Statistics DataFrame Shape:  (8915, 2651)
No Duplicated columns:  True
All Columns Data Types are Numeric:  True


In [18]:
df_complete_odds.tail()

DoubleChance__1X_count  DoubleChance__1X_mean  \
id                                                        
18220185                    17.0               1.058235   
18220186                    17.0               1.287059   
18220187                    17.0               1.204706   
18220188                    17.0               3.614118   
18220189                    17.0               1.161765   

          DoubleChance__1X_median  DoubleChance__1X_std  DoubleChance__1X_var  \
id                                                                              
18220185                     1.07              0.022146              0.000490   
18220186                     1.29              0.024941              0.000622   
18220187                     1.21              0.022671              0.000514   
18220188                     3.60              0.176070              0.031001   
18220189                     1.16              0.028556              0.000815   

          DoubleChance__1X_max  DoubleChance__1X_min  \
id                                                     
18220185                  1.09                  1.01   
18220186                  1.33                  1.22   
18220187                  1.23                  1.15   
18220188                  3.85                  3.20   
18220189                  1.25                  1.12   

          DoubleChance__1X_1st_quart  DoubleChance__1X_3rd_quart  \
id                                                                 
18220185                        1.04                        1.07   
18220186                        1.28                        1.30   
18220187                        1.19                        1.22   
18220188                        3.50                        3.74   
18220189                        1.15                        1.17   

          DoubleChance__1X_range  DoubleChance__1X_IQR  \
id                                                       
18220185                    0.08                  0.03   
18220186                    0.11                  0.02   
18220187                    0.08                  0.03   
18220188                    0.65                  0.24   
18220189                    0.13                  0.02   

          DoubleChance__12_count  DoubleChance__12_mean  \
id                                                        
18220185                    16.0               1.184375   
18220186                    16.0               1.315625   
18220187                    16.0               1.291875   
18220188                    16.0               1.148125   
18220189                    16.0               1.251250   

          DoubleChance__12_median  DoubleChance__12_std  DoubleChance__12_var  \
id                                                                              
18220185                     1.18              0.014592              0.000213   
18220186                     1.32              0.021282              0.000453   
18220187                     1.30              0.022574              0.000510   
18220188                     1.15              0.019397              0.000376   
18220189                     1.25              0.025265              0.000638   

          DoubleChance__12_max  DoubleChance__12_min  \
id                                                     
18220185                  1.21                  1.17   
18220186                  1.36                  1.28   
18220187                  1.32                  1.25   
18220188                  1.18                  1.11   
18220189                  1.29                  1.20   

          DoubleChance__12_1st_quart  DoubleChance__12_3rd_quart  \
id                                                                 
18220185                      1.1700                      1.1925   
18220186                      1.3000                      1.3225   
18220187                      1.2775                      1.3025   
18220188                      1.1400

In [19]:
df_complete_odds[df_complete_odds.filter(regex='count').columns.tolist()].describe()

,DoubleChance__1X_count,DoubleChance__12_count,DoubleChance__X2_count,DoubleChance-1stHalf__1X_count,DoubleChance-1stHalf__12_count,DoubleChance-1stHalf__X2_count,DoubleChance-2ndHalf__1X_count,DoubleChance-2ndHalf__12_count,DoubleChance-2ndHalf__X2_count,ExactGoalsNumber__0_count,ExactGoalsNumber__1_count,ExactGoalsNumber__2_count,ExactGoalsNumber__3_count,ExactGoalsNumber__4_count,ExactGoalsNumber__5_count,ExactGoalsNumber__6_count,ExactGoalsNumber__7_count,HTeamExactGoalsNumber__0_count,HTeamExactGoalsNumber__1_count,HTeamExactGoalsNumber__2_count,HTeamExactGoalsNumber__3_count,HTeamExactGoalsNumber__4_count,HTeamExactGoalsNumber__5_count,HTeamExactGoalsNumber__6_count,HTeamExactGoalsNumber__7_count,ATeamExactGoalsNumber__0_count,ATeamExactGoalsNumber__1_count,ATeamExactGoalsNumber__2_count,ATeamExactGoalsNumber__3_count,ATeamExactGoalsNumber__4_count,ATeamExactGoalsNumber__5_count,ATeamExactGoalsNumber__6_count,ATeamExactGoalsNumber__7_count,2ndHalfExactGoalsNumber__0_count,2ndHalfExactGoalsNumber__1_count,2ndHalfExactGoalsNumber__2_count,2ndHalfExactGoalsNumber__3_count,2ndHalfExactGoalsNumber__4_count,2ndHalfExactGoalsNumber__5_count,2ndHalfExactGoalsNumber__6_count,2ndHalfExactGoalsNumber__7_count,1stHalfExactGoalsNumber__0_count,1stHalfExactGoalsNumber__1_count,1stHalfExactGoalsNumber__2_count,1stHalfExactGoalsNumber__3_count,1stHalfExactGoalsNumber__4_count,1stHalfExactGoalsNumber__5_count,1stHalfExactGoalsNumber__6_count,1stHalfExactGoalsNumber__7_count,3W__1_count,3W__X_count,3W__2_count,3W1stHalf__1_count,3W1stHalf__X_count,3W1stHalf__2_count,3W2ndHalf__1_count,3W2ndHalf__X_count,3W2ndHalf__2_count,O/U__Over_count,O/U__Under_count,O/U1stHalf__Over_count,O/U1stHalf__Under_count,O/U2ndHalf__Over_count,O/U2ndHalf__Under_count,Corners1x2__1_count,Corners1x2__X_count,Corners1x2__2_count,TeamToScoreFirst__1_count,TeamToScoreFirst__X_count,TeamToScoreFirst__2_count,H/A_1_count,H/A_2_count,HTeamScoreaGoal__No_count,HTeamScoreaGoal__Yes_count,ATeamScoreaGoal__No_count,ATeamScoreaGoal__Yes_count,BothTeamsToScore__No_count,BothTeamsToScore__Yes_count,BothTeamsToScore-1stHalf__No_count,BothTeamsToScore-1stHalf__Yes_count,BothTeamsToScore-2ndHalf__No_count,BothTeamsToScore-2ndHalf__Yes_count,Odd/Even__Even_count,Odd/Even__Odd_count,Odd/Even1stHalf__Even_count,Odd/Even1stHalf__Odd_count,CornersOU__Over_count,CornersOU__Under_count,AH__1_count,AH__2_count,AHFirstHalf__1_count,AHFirstHalf__2_count,Total-H__Over_count,Total-H__Under_count,Total-A__Over_count,Total-A__Under_count,WinToNil__1_count,WinToNil__2_count,WinBothHalves__1_count,WinBothHalves__2_count,TeamToScoreLast__1_count,TeamToScoreLast__2_count,TeamToScoreLast__X_count,HighestScoringHalf__1st Half_count,HighestScoringHalf__2nd Half_count,HighestScoringHalf__X_count,CleanSheet-H__Yes_count,CleanSheet-H__No_count,CleanSheet-A__Yes_count,CleanSheet-A__No_count,ToWinEitherHalf__1_count,ToWinEitherHalf__2_count,CS__0:0_count,CS__1:0_count,CS__0:1_count,CS__2:0_count,CS__0:2_count,CS__3:0_count,CS__0:3_count,CS__4:0_count,CS__0:4_count,CS__5:0_count,CS__0:5_count,CS__6:0_count,CS__0:6_count,CS__7:0_count,CS__0:7_count,CS__1:1_count,CS__2:1_count,CS__1:2_count,CS__3:1_count,CS__1:3_count,CS__4:1_count,CS__1:4_count,CS__5:1_count,CS__1:5_count,CS__6:1_count,CS__1:6_count,CS__2:2_count,CS__3:2_count,CS__2:3_count,CS__4:2_count,CS__2:4_count,CS__5:2_count,CS__2:5_count,CS__6:2_count,CS__2:6_count,CS__3:3_count,CS__4:3_count,CS__3:4_count,CS__4:4_count,CS__5:3_count,CS__3:5_count,CS1stHalf__0:0_count,CS1stHalf__1:0_count,CS1stHalf__0:1_count,CS1stHalf__2:0_count,CS1stHalf__0:2_count,CS1stHalf__3:0_count,CS1stHalf__0:3_count,CS1stHalf__4:0_count,CS1stHalf__0:4_count,CS1stHalf__5:0_count,CS1stHalf__0:5_count,CS1stHalf__6:0_count,CS1stHalf__0:6_count,CS1stHalf__7:0_count,CS1stHalf__0:7_count,CS1stHalf__1:1_count,CS1stHalf__2:1_count,CS1stHalf__1:2_count,CS1stHalf__3:1_count,CS1stHalf__1:3_count,CS1stHalf__4:1_count,CS1stHalf__1:4_count,CS1stHalf__5:1_count,CS1stHalf__1:5_count,CS1s

In [20]:
df_complete_odds.isna().sum()

DoubleChance__1X_count                        5
DoubleChance__1X_mean                        10
DoubleChance__1X_median                      10
DoubleChance__1X_std                         54
DoubleChance__1X_var                         54
DoubleChance__1X_max                         10
DoubleChance__1X_min                         10
DoubleChance__1X_1st_quart                   10
DoubleChance__1X_3rd_quart                   10
DoubleChance__1X_range                       10
DoubleChance__1X_IQR                         10
DoubleChance__12_count                        5
DoubleChance__12_mean                        12
DoubleChance__12_median                      12
DoubleChance__12_std                         53
DoubleChance__12_var                         53
DoubleChance__12_max                         12
DoubleChance__12_min                         12
DoubleChance__12_1st_quart                   12
DoubleChance__12_3rd_quart                   12
DoubleChance__12_range                  

In [22]:
df_complete_odds.to_csv('../../Data/Modeling_Before_Preparation/Odds_data_complete_v01.csv')

### *extra test - have all odds types been considered?*

In [23]:
# List of all odds types considered in this process
considered_columns = ['^3W','^O/U','^DoubleChance','^Corners1x2','^TeamToScoreFirst_','^H/A_','TeamsToScoreAGoal','ScoreaGoal','BothTeamsToScore', 'Odd/Even', '^HT/FT_', 'CornersOU', 'ACornersO/U','HCornersO/U', '^AH', '^TeamTG', '^RES/TG', '^Total-(H|A)', '^(WinToNil|WintoNil)', '^(A|H)winbothhalves_', 'TeamToScoreLast_', '^(ATeam|HTeam|)HighestScoringHalf_', '^CleanSheet', '^ToWinEitherHalf', 'First10minWinner', 'ExactGoalsNumber', 'WinningMargin', '^1stPlayerBooked', 'ToScoreInBothHalvesByTeams', '^CS', '^HT/FTDouble', 'GoalLine', '^ToMissAPenalty', '^ToScoreAPenalty', 'OwnGoal', 'PlayerAssists_', '^LateGoal_', '^TimeOfFirstCard', '^LastMatchCorner_', '^TotalGoalMinutes', '^1stHalfHnd_', '^FirstCardReceived', '^FirstMatchCorner_', '^PlayerPasses_', '^DrawNoBet', '^Specials_', '^CornerHnd_', '^TimeOf1stTeamGoal_', '^AsianTotalCards_', 'CardHnd_', '^TeamShotsonTarget_', '^PlayerShots_', '^Multicorners_', '^MatchShots_', '^HalftimeRES/TG_', '^PlayerTackles_', '^1stHalfRES/TG_', '^TeamShots_', '^WinBothHalves_', '^MatchShotsonTarget_', '^BothTeamsToReceive', '^TeamOffsides_', '^ToScoreInHalf_', '^2-WayCorners_', '^CornersRace_', '^NumberofCards_', 'FirstCorner_', '^Alternative1stHalfAH', '^EarlyGoal_', '^CornerMatchBet', '^AsianTotalCorners', '^TimeOf1stGoalBrackets_', '^10MinuteRES_', '^NumberofGoalsinMatch', '^MatchOffsides_', '^SetBetting_', '^1stGoalMethod_', '^AlternativeAH', '^MatchTackles_', '^1stHalfAH_', '^TeamCards_', '^PlayerShotsonTarget', '^1stHalfAsianCorners_', '^1stHalfCorners_', '^H/A\\(2ndSet\\)', '^BothTeamstoScore1stHalf-2ndHalf_', '^LastCorner_', '^ATeamTG\\(1stHalf\\)_', '^CornersAH_', 'TotalCorners_', '^ARedCardintheMatch_', '^TeamPerformances_', '^H/A\\(1stSet\\)', '^AlternativeTG_', 'APenaltyintheMatch', 'FullTimeRES–EnhancedPrices', '1stHalfWinner', '^10O/U_', '^Alternative.*HndRES', '^HndRES.*__(1|2|X|Tie)$', '^HndRES_.*__(1|2|X|Tie) : (\\+|\\-)\d.\d', 'HndRES', '^(Team|Total)Corners', '^Numberofsets']
# Subtract considered columns from all the columns of the initial DataFrame in order to see if the process was meticulously done
all_cols = leagues_odds_df.columns.tolist()
print('Length of df columns: ', len(all_cols))
for i in considered_columns:
    all_cols = list(set(all_cols) - set(leagues_odds_df.filter(regex=i).columns.tolist()))
print('Lenght of not-considered columns: ', len(all_cols), '\nList of not-considered columns: ', all_cols)

Length of df columns:  84055
Lenght of not-considered columns:  0 
List of not-considered columns:  []
